In [14]:
from dolfin import *
import matplotlib.pyplot as plt

In [15]:
# Optimization options for the form compiler
parameters["form_compiler"]["cpp_optimize"] = True
ffc_options = {"optimize": True, \
               "eliminate_zeros": True, \
               "precompute_basis_const": True, \
               "precompute_ip_const": True}

In [16]:
# Create mesh and define function space
mesh = UnitCubeMesh(24, 16, 16)
V = VectorFunctionSpace(mesh, 'Lagrange', 1)

In [17]:
# Mark boundary subdomains
left = CompiledSubDomain("near(x[0], side) && on_boundary", side=0.0)
right = CompiledSubDomain("near(x[0], side) && on_boundary", side=1.0)

In [18]:
# Define Dirichlet boundary values (x = 0 or x = 1)
c = Constant((0, 0, 0))
r = Expression(("scale*0.0",
                "scale*(y0 + (x[1] - y0)*cos(theta) - (x[2] - z0)*sin(theta) - x[1])",
                "scale*(z0 + (x[1] - y0)*sin(theta) + (x[2] - z0)*cos(theta) - x[2])"),
                scale = 0.5, y0 = 0.5, z0 = 0.5, theta = pi/3, degree=2)

In [19]:
bcLeft = DirichletBC(V, c, left)
bcRight = DirichletBC(V, r, right)
bcs = [bcLeft, bcRight]

In [20]:
# Define functions
du = TrialFunction(V)    # incremental displacement
v = TestFunction(V)      # test function
u = Function(V)          # displacement from previous iteration
B = Constant((0.0, -0.5, 0.0))    # body force per unit volume
T = Constant((0.1, 0.0, 0.0))

#In place of Constant, it is also possible to use as_vector,
# e.g. B = as_vector( [0.0, -0.5, 0.0] ). 
# The advantage of Constant is that its values can be changed 
# without requiring re-generation and re-compilation of C++ code. 
# On the other hand, using as_vector can eliminate some 
# function calls during assembly.

# B = as_vector([0.0, -0.5, 0.0])
# T = as_vector([0.1, 0.0, 0.0]) 

In [21]:
# The kinematic quantities involved in the model are defined using UFL syntax:
# Kinematatics
d = len(u)                   # spatial dimension
F = Identity(d) + grad(u)    # deformation gradient
C = F.T * F                  # right cauchy-green tensor

# Invariants of deformation tensors
Ic = tr(C)
J = det(F)

In [22]:
# The material parameters are set and the strain energy density and the total
# potential energy are defined, again using UFL syntax.

# Elasticity parameters
E, nu = 10.0, 0.3
mu_ = Constant(E / (2*(1+ nu)))
lambda_ = Constant(E * nu / ((1 + nu)*(1 - 2*nu)))

# Stored strain energy density (compressible neo-Hookean model)
psi = 0.5 * mu_ * (Ic - 3) - mu_ * ln(J) + 0.5*lambda_ * (ln(J))**2

# Total potential energy = (internal energy - external potential)*dx
Pi = psi*dx - dot(B, u)*dx - dot(T, u)*ds

In [23]:
# Compute first variation of Pi 
# namely, directional derivative about u in the direction of v
F = derivative(Pi, u, v)
# Compute Jacobian of F
Jac_of_F = derivative(F, u, du)

In [24]:
# Solve variational problem
solve(F == 0, u, bcs, J=Jac_of_F, form_compiler_parameters=ffc_options)

In [25]:
# Save solution in VTK format
file = File("displacement.pvd");
file << u;